In [7]:
import pandas as pd
from pathlib import Path

In [58]:
def read_results(alg: str) -> pd.DataFrame:
  root_dir = Path.cwd().parent
  results_dir = root_dir / "results" / alg 

  history_path = results_dir / "history.csv"
  history_df = pd.read_csv(history_path)
  history_df['timestamp'] = pd.to_datetime(history_df['timestamp'])
  return history_df

In [ ]:
def count_time(history_df: pd.DataFrame) -> int:
  from_time = history_df['timestamp'].iloc[0]
  to_time = history_df['timestamp'].iloc[-1]
  diff = to_time - from_time
  return diff

In [ ]:
history_df = read_results('mab')
time_diff = count_time(history_df)
print(f'Time diff {time_diff}')

0 days 00:04:53
